IMPORTS

In [1]:
from google.colab import drive
from geopy.geocoders import Nominatim
import pandas as pd
import numpy as np

LOADING DATA

In [2]:
drive.mount('/content/gdrive')
%cd gdrive/My Drive/Colab Notebooks/IC23/

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive/Colab Notebooks/IC23


In [52]:
#! git clone https://github.com/osullik/ic23050.git
#! git pull

In [3]:
orig_data = pd.read_csv("https://raw.githubusercontent.com/osullik/ic23050/main/L4%20Washington%20Fatal%20Crash%20Files%20-%20Washington%20Traffic%20Safety%20Commission/Data_Washington%20Fatal%20Crash%20Survey.csv")

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (21,22,23,301,303) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
orig_data.head()

,year,case,par,repjur,crash_dt,crash_tm,accday,accmon,holiday,county,...,nmdistract6,lab,isedtcase,crf1,crf2,crf3,CoRoadName,CoMP,IntCoRoadName,IntCoMP
0,2017,1,E628946,2.0,01/01/2017,2:12,1,1,1.0,61,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2017,2,E627989,26.0,01/02/2017,17:14,2,1,1.0,35,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2017,4,3747633,263.0,01/01/2017,18:47,1,1,1.0,33,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2017,5,E628691,4.0,01/01/2017,3:50,1,1,1.0,63,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2017,6,3746306,263.0,01/05/2017,9:53,5,1,0.0,33,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


REVERSE GEOCODING

In [11]:
locator = Nominatim(user_agent="myGeocoder")

In [20]:
# Input: lat and long coordinates
# Output: dictionary containing 'address' -> 'postcode' and other address fields
def reverse_geo(x, y):
  try:
    return locator.reverse((x,y)).raw
  except ValueError:
    print(x, y)

In [41]:
# Input: geo dictionary
# Output: zipcode
def get_zipcode(geodict):
  try:
    return geodict['address']['postcode']
  except KeyError:
    return np.nan

In [13]:
#x, y = 40.7128, -74.0060
#reverse_geo(x, y)

{'place_id': 226179173,
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
 'osm_type': 'way',
 'osm_id': 575213527,
 'lat': '40.71273945',
 'lon': '-74.00593904130275',
 'display_name': 'New York City Hall, 260, Broadway, Lower Manhattan, Manhattan Community Board 1, Manhattan, New York County, City of New York, New York, 10000, United States',
 'address': {'amenity': 'New York City Hall',
  'house_number': '260',
  'road': 'Broadway',
  'quarter': 'Lower Manhattan',
  'neighbourhood': 'Manhattan Community Board 1',
  'suburb': 'Manhattan',
  'county': 'New York County',
  'city': 'City of New York',
  'state': 'New York',
  'ISO3166-2-lvl4': 'US-NY',
  'postcode': '10000',
  'country': 'United States',
  'country_code': 'us'},
 'boundingbox': ['40.712445', '40.7130254', '-74.0064455', '-74.0055687']}

In [43]:
orig_data['zipcode'] = orig_data.apply(lambda row: get_zipcode(reverse_geo(row['y'], row['x'])), axis=1)

In [51]:
orig_data.to_csv('/content/gdrive/My Drive/Colab Notebooks/IC23/Data_Washington_Fatal_Crash_Survey_with_zip.csv')

AUGMENTING WITH OSM DATA

In [56]:
!pip install pyrosm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 40.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached cykhash-2.0.1-cp38-cp38-linux_x86_64.whl
  Using cached pyrobuf-0.9.3-cp38-cp38-linux_x86_64.whl
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 58.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 75.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 60.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 85.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 66.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.6/96.6 KB 9.1 MB/s eta 0:00:00
  Created wheel for pyrosm: filename=pyrosm-0.6.1-cp38-cp38-linux_x86_64.whl size=8137830 sha256=e

In [3]:
from pyrosm import OSM
from pyrosm import get_data
from pyrosm.data import sources

/usr/local/lib/python3.8/dist-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/pyrosm/utils/__init__.py:8: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/late

In [4]:
data_with_zip = pd.read_csv('https://raw.githubusercontent.com/osullik/ic23050/main/L4%20Washington%20Fatal%20Crash%20Files%20-%20Washington%20Traffic%20Safety%20Commission/Data_Washington_Fatal_Crash_Survey_with_zip.csv')

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (22,23,24,302,304) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
# Grab OSM data from pyrosm
fp = get_data("washington") # AKA sources.north_america.usa.washington also at '/content/gdrive/My Drive/Colab Notebooks/IC23/washington-latest.osm.bz2'

# Initialize the OSM parser object
osm = OSM(fp)

In [ ]:
# Read all drivable roads
drive_net = osm.get_network(network_type="driving")
drive_net.plot()

In [ ]:
drive_net.head(2)

AUGMENT ZIPCODES WITH DEMOGRAPHICS

In [5]:
!pip install uszipcode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.5/121.5 KB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 KB 7.0 MB/s eta 0:00:00


In [8]:
from uszipcode import SearchEngine  # data.census.gov

In [13]:
def augment_zip(zipcode):
  aug_zip = SearchEngine().by_zipcode(zipcode)
  return aug_zip.zipcode, aug_zip.major_city, aug_zip.population  # TODO edit fields to be what we want to add to the data listed here: https://uszipcode.readthedocs.io/

print(augment_zip(11790))

('11790', 'Stony Brook', 18511)
